## 변수 10개 추출하기(~04/24)
* 총 접속 일수	total_cnt_dt (표준화 X)
* 총 플레이시간	total_play_time
* NPC 사냥 일반 경험치	total_npc_exp
* NPC 사냥 홍문 경험치	total_npc_hongmun
* 퀘스트 일반 경험치	total_quest_exp
* 퀘스트 홍문 경험치	total_quest_hongmun
* 아이템 홍문 경험치	total_item_hongmun
* 총 재화 획득량	total_get_money
* 결투 참여 횟수	total_duel_cnt
* 결투 승리 횟수	total_duel_win

In [1]:
import pandas as pd
import numpy as np

In [2]:
activity = pd.read_csv('../../../final_data_rev/train/train_activity.csv')

In [3]:
activity.shape

(440323, 38)

In [5]:
# 유저 아이디 추출
acc_id_uniq = activity.acc_id.unique()
acc_id_uniq.shape

(100000,)

In [23]:
# structured data를 만들 데이터프레임 df 선언
df = pd.DataFrame({'acc_id':acc_id_uniq})
df.shape

(100000, 1)

In [24]:
var_name_list = ['play_time','npc_exp', 'npc_hongmun', 'quest_exp', 'quest_hongmun','item_hongmun', 'get_money', 'duel_cnt', 'duel_win']

In [25]:
total_cnt_dt = activity.groupby('acc_id').cnt_dt.sum()
df = df.merge(total_cnt_dt, left_on=['acc_id'], right_on=['acc_id'])

In [26]:
# 집계(aggrigate) 할 때 사용하는 함수
def get_total_agg_func(values, val_min):
    play_wk = len(values) # 총 접속한 주
    non_play_wk = 8 - play_wk # 접속하지 않은 주의 수
    
    calibrate_val = non_play_wk * val_min # 보정해줄 값 = 접속하지 않은 주의 수 * 최소값
    
    val_sum = 0
    for value in values:
        val_sum += value
        
    val_total = val_sum + calibrate_val # 값들의 총 합 + 보정할 값
    return val_total

In [27]:
# df에 변수를 추가하는 함수
def add_variable(df, var_name_list):
    
    for var_name in var_name_list:
        groupby = activity.groupby('acc_id')['{}'.format(var_name)] # 해당하는 변수에 대해 groupby 해줌
        val_min = activity['{}'.format(var_name)].min() # 최소값을 변수에 저장
        agg = groupby.agg(get_total_agg_func, val_min=val_min) # 각 그룹마다 총합을 구해줌
        df = df.merge(agg, left_on='acc_id', right_on='acc_id') # df에 merge 진행
    return df

In [28]:
df = add_variable(df, var_name_list)

In [29]:
df.head()

,acc_id,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,get_money,duel_cnt,duel_win
0,3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...,9,0.792270,7.773991,-2.212823,9.058999,4.842328,-2.450829,-0.188121,-0.289344,-0.330606
1,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,15,-5.262302,-1.854139,-2.326426,-1.998941,-3.901330,-2.450829,-0.188343,-0.289344,-0.330606
2,fa883ca7505082114c4024052354f1fb416f6bae26ed06...,6,-5.289857,-1.854992,-2.326426,-2.003387,-3.901330,-2.450829,-0.188343,-0.289344,-0.330606
3,d094b6b1c5d0a147eaae3e37b256894def52de39c6eca3...,3,-4.025387,-0.247012,-2.178636,0.215844,-1.570812,-2.357342,-0.188287,-0.289344,-0.330606
4,38e7088d64485baba2968be8ad56f5b8abeced8ccd95f1...,50,0.243092,-1.854992,-0.823104,-2.003387,-3.012494,-1.557510,-0.187585,-0.289344,-0.330606


In [32]:
cols = dict()
for var_name in var_name_list:
    cols['{}'.format(var_name)] = 'total_{}'.format(var_name)
cols['cnt_dt'] = 'total_cnt_dt'

In [33]:
# column 이름 변경
df = df.rename(index=str, columns=cols)
df.head()

,acc_id,total_cnt_dt,total_play_time,total_npc_exp,total_npc_hongmun,total_quest_exp,total_quest_hongmun,total_item_hongmun,total_get_money,total_duel_cnt,total_duel_win
0,3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...,9,0.792270,7.773991,-2.212823,9.058999,4.842328,-2.450829,-0.188121,-0.289344,-0.330606
1,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,15,-5.262302,-1.854139,-2.326426,-1.998941,-3.901330,-2.450829,-0.188343,-0.289344,-0.330606
2,fa883ca7505082114c4024052354f1fb416f6bae26ed06...,6,-5.289857,-1.854992,-2.326426,-2.003387,-3.901330,-2.450829,-0.188343,-0.289344,-0.330606
3,d094b6b1c5d0a147eaae3e37b256894def52de39c6eca3...,3,-4.025387,-0.247012,-2.178636,0.215844,-1.570812,-2.357342,-0.188287,-0.289344,-0.330606
4,38e7088d64485baba2968be8ad56f5b8abeced8ccd95f1...,50,0.243092,-1.854992,-0.823104,-2.003387,-3.012494,-1.557510,-0.187585,-0.289344,-0.330606


In [34]:
df.to_csv('variable10.csv')